In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=bdcabc27934d07b9d14a6c0cb2bd35e61c0cf95fd8315215533990cc1704faa7
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 6.4 MB/s eta 0:00:00


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import csv
import numpy as np
import random
from transformers import AutoModelForCausalLM, AutoTokenizer
import evaluate

In [ ]:
codeLines = []
with open('/data.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        codeLines.append(row)
codeLines = np.array(codeLines, dtype=np.str_)
codeLines[0:5]

array([['case IntelIntDir(d) => \\int %d\\" format d"'],
       ['case IntelAscizDir(s) => \\asciz \\\\\\"%s\\\\\\"\\" format escapeChars(s)"'],
       ['case IntelSyntaxDir(s) => \\intel_syntax %s\\" format s"'],
       ['case IntelTextDir => \\text\\""'],
       ['private def coreEightFormat(s: String, end: String)(implicit size: IntelSize): String =']],
      dtype='<U87')

In [ ]:
# Split line into prefix, middle, and suffix
def split_code(line):
    cursor_position = random.randint(0, len(line) - 5)
    prefix = line[:cursor_position]

    middle_end = random.randint(cursor_position + 5, len(line))
    middle = line[cursor_position:middle_end]

    suffix = line[middle_end:]

    return prefix, middle, suffix

In [ ]:
DATA_LEN = 30

data = []
for _ in range(DATA_LEN):
    line = random.choice(codeLines)  # Randomly choose a line
    prefix, middle, suffix = split_code(line[0])
    data.append((prefix, middle, suffix))

print(len(data))
data[0:5]

30


[('def allocate(tmpReg: Register): IntelAdd', 'r = {', ''),
 ('cas', 'e IdentReg(id, t) =>', ' identPrefix'),
 ('', 'case _ => \\\\""', ''),
 ('case (id, ', 's) =>', ''),
 ('priv',
  'ate def genFuncArgsPushPopList',
  '(args: Int): List[IntelRegister] =')]

In [ ]:
checkpoint = "bigcode/tiny_starcoder_py"
device = "cpu"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)
outputs = []

for pref, _, suf in data:
  input_text = f"<fim_prefix>{pref}<fim_suffix>{suf}<fim_middle>"
  inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
  output = tokenizer.decode(model.generate(inputs, pad_token_id=0, max_new_tokens=30)[0])
  middle = output.split("<fim_middle>")[1]
  outputs.append(middle)

print(len(outputs))


30


In [ ]:
outputs = list(map(lambda s: s.replace("<|endoftext|>", "").rstrip('\n'), outputs))

for i in range(DATA_LEN):
  pref, mid, suf = data[i]
  print(f">>> Prefix {i}:")
  print(pref)
  print(f"### Prediction {i}:")
  print(outputs[i])
  print(f'### Actual {i}:')
  print(mid)
  print(f">>> Sufix {i}:")
  print(suf)
  print("##############")
  print("##############")

>>> Prefix 0:
def allocate(tmpReg: Register): IntelAdd
### Prediction 0:
er(tmpReg)
### Actual 0:
r = {
>>> Sufix 0:

##############
##############
>>> Prefix 1:
cas
### Prediction 1:
k_prefix(self,
### Actual 1:
e IdentReg(id, t) =>
>>> Sufix 1:
 identPrefix
##############
##############
>>> Prefix 2:

### Prediction 2:

### Actual 2:
case _ => \\""
>>> Sufix 2:

##############
##############
>>> Prefix 3:
case (id, 
### Prediction 3:
id, 1)
### Actual 3:
s) =>
>>> Sufix 3:

##############
##############
>>> Prefix 4:
priv
### Prediction 4:
er
### Actual 4:
ate def genFuncArgsPushPopList
>>> Sufix 4:
(args: Int): List[IntelRegister] =
##############
##############
>>> Prefix 5:
private val callee: List[IntelRegister] = List(R
### Prediction 5:
egister(R15, R15.R15, R15.R15, R15.R15
### Actual 5:
12, R13, R14, 
>>> Sufix 5:
R15)
##############
##############
>>> Prefix 6:
case IntelRet => 
### Prediction 6:
" + str(ret))
        print("Case IntelRet => 0")
        print("Case IntelRet 

In [ ]:
exact_match = 0

for (pref, mid, suf), output in zip(data, outputs):
  if mid == output:
    exact_match += 1

print(exact_match / DATA_LEN)

0.0


In [ ]:
# Chrf
actual_middles = [mid for _, mid, _ in data]
chrf = evaluate.load("chrf")
results = chrf.compute(predictions=outputs, references=actual_middles)
print(results)

{'score': 7.519379257344762, 'char_order': 6, 'word_order': 0, 'beta': 2}


In [ ]:
# Bleu
bleu = evaluate.load("sacrebleu")
results = bleu.compute(predictions=outputs, references=actual_middles)
print(results)


# Rouge
rouge = evaluate.load("rouge")
results = rouge.compute(predictions=outputs, references=actual_middles)
print(results)

{'score': 0.5846970281118166, 'counts': [31, 4, 0, 0], 'totals': [380, 351, 326, 305], 'precisions': [8.157894736842104, 1.1396011396011396, 0.15337423312883436, 0.08196721311475409], 'bp': 1.0, 'sys_len': 380, 'ref_len': 131}
{'rouge1': 0.02333333333333333, 'rouge2': 0.0, 'rougeL': 0.02, 'rougeLsum': 0.02333333333333333}
